In [80]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

from cell.utils import train_val_test_split, link_prediction_performance
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics

## Load Graph

In [70]:
_A_obs = load_npz(f'./data/CORA-ML.npz')

val_share = 0.1
test_share = 0.05
seed = 10

train_graph, val_ones, val_zeros, test_ones, test_zeros = train_val_test_split(_A_obs,
                                                                               val_share,
                                                                               test_share,
                                                                               seed)

## Edge Overlap Criterion

In [71]:
# initialize model with EO-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[EdgeOverlapCriterion(invoke_every=10, EO_limit=.5)])

In [72]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-7})

Step:  10/200 Loss: 5.99155 Edge-Overlap: 0.035 Total-Time: 5
Step:  20/200 Loss: 4.07789 Edge-Overlap: 0.280 Total-Time: 10
Step:  30/200 Loss: 3.32820 Edge-Overlap: 0.448 Total-Time: 16
Step:  40/200 Loss: 2.97465 Edge-Overlap: 0.547 Total-Time: 21


In [77]:
generated_graph = model.sample_graph()

In [78]:
compute_graph_statistics(generated_graph)

{'d_max': 161.0,
 'd_min': 1.0,
 'd': 4.8284697508896794,
 'LCC': 2804,
 'wedge_count': 73803.0,
 'claw_count': 1145656.0,
 'triangle_count': 1374,
 'square_count': 6530.0,
 'power_law_exp': 1.8126554334133969,
 'gini': 0.4426437755992749,
 'rel_edge_distr_entropy': 0.9526665796124112,
 'assortativity': -0.08150885625778884,
 'clustering_coefficient': 0.003597938648250435,
 'cpl': 5.221674895650144}

In [79]:
compute_graph_statistics(train_graph)

{'d_max': 209.0,
 'd_min': 1.0,
 'd': 4.8284697508896794,
 'LCC': 2810,
 'wedge_count': 95802.0,
 'claw_count': 2301166.0,
 'triangle_count': 2867,
 'square_count': 14969.0,
 'power_law_exp': 1.8585544293441962,
 'gini': 0.4855507830860135,
 'rel_edge_distr_entropy': 0.9408725499709523,
 'assortativity': -0.07557187056300232,
 'clustering_coefficient': 0.0037376703810155375,
 'cpl': 5.672324214617732}

## Validation Criterion

In [81]:
# initialize model with LP-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[LinkPredictionCriterion(invoke_every=2,
                                                val_ones=val_ones,
                                                val_zeros=val_zeros,
                                                max_patience=3)])

In [82]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-6})

Step:   2/200 Loss: 7.92690 ROC-AUC Score: 0.619 Average Precision: 0.613 Total-Time: 0
Step:   4/200 Loss: 7.72117 ROC-AUC Score: 0.673 Average Precision: 0.683 Total-Time: 1
Step:   6/200 Loss: 7.29555 ROC-AUC Score: 0.710 Average Precision: 0.727 Total-Time: 2
Step:   8/200 Loss: 6.68050 ROC-AUC Score: 0.744 Average Precision: 0.763 Total-Time: 3
Step:  10/200 Loss: 6.00421 ROC-AUC Score: 0.789 Average Precision: 0.807 Total-Time: 4
Step:  12/200 Loss: 5.40855 ROC-AUC Score: 0.834 Average Precision: 0.850 Total-Time: 5
Step:  14/200 Loss: 4.97076 ROC-AUC Score: 0.865 Average Precision: 0.879 Total-Time: 6
Step:  16/200 Loss: 4.64708 ROC-AUC Score: 0.880 Average Precision: 0.895 Total-Time: 7
Step:  18/200 Loss: 4.36183 ROC-AUC Score: 0.888 Average Precision: 0.903 Total-Time: 8
Step:  20/200 Loss: 4.11109 ROC-AUC Score: 0.895 Average Precision: 0.910 Total-Time: 9
Step:  22/200 Loss: 3.91517 ROC-AUC Score: 0.903 Average Precision: 0.916 Total-Time: 10
Step:  24/200 Loss: 3.74309 ROC

In [83]:
link_prediction_performance(scores_matrix=model._scores_matrix, val_ones=test_ones, val_zeros=test_zeros)

(0.9371172291631334, 0.9404623466909817)